# AXI Stream Implementation of RSA

This code builds off what was learned in Example #3 using chars. In this version, the conversion of the string to an array was moved out of the encryption/decryption functions and into its own dedicated functions. The first half of the code is the supporting python code and an example test of converting a string to an array and back. For the second half of the code, there is the beginning of the implementation for the axi stream functionality. The supporting HLS code for this is located in the file RSA_encryption.cpp which performs the calulcation on the input_buffer integer. 

The cpp file still needs to be tested, so the overlay part of this notebook is incomplete/non-functional as of now (3/2).

In [21]:
# Supporting code for implementing
import random
import math

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a
# Function to generate public key
def pubKeyGen(p, q):
    #define phi with (p,q)
    phi = (p-1)*(q-1)
    # calculate/return a value for e (public key)
    e = 2
    while (e < phi):
        if(gcd(e, phi) == 1):
            break
        else:
            e=e+1
    return e

# Functions to generate public/private keys
def generate_keypair(prime1, prime2):

    n = prime1 * prime2
    phi = (prime1 - 1) * (prime2 - 1)

    # Generate a public key value e from the primes
    e = pubKeyGen(prime1, prime2)

    # Compute the private key d.
    d = pow(e, -1, phi)
 
    # Return the public and private keypairs
    return ((e, n), (d, n))

# Function to convert the message to an array of integers.
def prepInput(message):
    intArrString = [ord(char) for char in message]
    return intArrString
# Function to reverse the action above and return a string from an array of integers.
def convOutput(orgMsg):
    stringFrInt = [chr(char) for char in orgMsg]
    return ''.join(stringFrInt)

In [22]:
# Test of converting string to a list of int - Passed!
inStr = "Hello"
inStr = prepInput(inStr)
print(inStr)
inStr =convOutput(inStr)
print(inStr)

[72, 101, 108, 108, 111]
Hello


## Overlay Code

In [ ]:
# Implementation Framework for AXI Stream (adapted from HW4 until Bitstream is working)

from pynq import Overlay
# Bit file from Vivado
ol = Overlay('')

dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

hls_ip = ol.example_0 

CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x81)

In [ ]:
from pynq import allocate
import numpy as np

# Adapted function to allocate the input and output buffers based on the amount of chars
def initilizeIO(message):

    input_buffer = allocate(shape=(len(message),), dtype=np.uint64)
    output_buffer = allocate(shape=(len(message),), dtype=np.uint64)
    return output_buffer, input_buffer



Note: this is a concept to show how I am planning on implementing this algorithm from HW. This has not been tested yet, and will likely change drastically over the course of this project.

In [ ]:
# Example of using this code

# First provide the input string (later this will be input from user, but 'hello' is easier to test)
input_message = "hello"

# Intilize the input/output buffers from the message
output_buffer, input_buffer = initilizeIO(input_message)

# Convert String
input_buffer = prepInput(input_message)

# Send to PL for encryption/decryption
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
dma_send.wait()
dma_recv.wait()

# Once finished, convert back to string and print
output_message = convOutput(output_buffer)
print(output_message)